Código para baixar os endereços dos arquivos na página

In [1]:
import os

def baixar_arquivo(url, local_filename=None):
    """Baixa e salva o arquivo de uma url."""
    # Se o nome do arquivo não foi passado como parâmetro, extrair da url.
    if local_filename is None:
        local_filename = os.path.join("./data", url.split('/')[-1])

    #verifica se arquivo já foi baixado
    if os.path.exists(local_filename):
        print(f'Arquivo {local_filename} já baixado!')
        return None
    else:
        print('Solicitando arquivo do servidor.')
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(local_filename, 'wb') as f:
            print(f"Baixando arquivo da url: {url}")
            for chunk in response.iter_content(chunk_size=8192): 
                f.write(chunk)
            return "Arquivo baixado!"



In [2]:
def read_page(url):
    print(f'Fazendo requisição GET para o link: {url}')
    response = requests.get(url)
    response.raise_for_status() #Levanta uma exceção caso haja erro na requisição.
    
    if response.status_code == 200:
        content = response.text
        return content

Informar a página

In [3]:
import requests
from bs4 import BeautifulSoup
import time

TIME_TO_SLEEP = 120

url = 'https://aplicacoes.mds.gov.br/snas/vigilancia/index2.php'

content = read_page(url)
soup = BeautifulSoup(content, 'html.parser')

Fazendo requisição GET para o link: https://aplicacoes.mds.gov.br/snas/vigilancia/index2.php


Informar a marcação em que os endereços dos arquivos estão no HTML

In [4]:
 section = soup.find('body')
 links=section.find_all('a')
 

Todos os links da página

In [5]:
len(links)

171

In [6]:
links[10]

<a href="https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/8%20-%20POSTO%20DE%20CADASTRAMENTO.rar">POSTO DE CADASTRAMENTO</a>

Identificar apenas os links que possuem o termo CRAS

In [7]:
CRAS=[]

for item in links:
    link = item['href']
    term_link = link[75:83]
    if term_link == 'RMA_CRAS':
        CRAS.append(link)
        


In [8]:
CRAS

['https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2023_divulga%C3%A7%C3%A3o_030423.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2022_divulga%C3%A7%C3%A3o_27_03_23.xlsx',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2021_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_Criterios_2020_divulga%C3%A7%C3%A3o(2).zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_NovoCriterios_2019_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_NovoCriterios_2018_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/files/RMA_CRAS_NovoCriterios_2017_divulga%C3%A7%C3%A3o.zip',
 'https://aplicacoes.mds.gov.br/sagi/dicivip_datain/ckfinder/userfiles/f

Pegar apenas o link de 2022 - segundo arquivo

In [9]:
for item in CRAS[1:2]:
    if baixar_arquivo(item):
        print('Esperando 120s para solicitar próximo download...')
        time.sleep(TIME_TO_SLEEP)
    else:
        print("Link para download não encontrado.")         

Arquivo ./data\RMA_CRAS_Criterios_2022_divulga%C3%A7%C3%A3o_27_03_23.xlsx já baixado!
Link para download não encontrado.


Descompactar o arquivo - ZIP na pasta data

In [10]:
import os
import zipfile

#caminho = os.getcwd()
caminho = 'data'
arquivos = os.listdir(caminho)

for arquivo in arquivos:
    if arquivo.endswith('.zip'):
        caminho_completo = os.path.join(caminho, arquivo)

        with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
            zip_ref.extractall(caminho)

In [11]:
import pandas as pd 
caminho

'data'

In [12]:
#arquivos=[]
#for file in os.listdir(caminho):
#    if file.endswith(".xlsx"):
#        print(f'Carregando arquivo {file} ...')
#        arquivos.append(pd.read_excel(os.path.join(caminho,file), sheet_name="Base tratada"))

Buscar apenas os arquivos terminados em XLSX na pasta data

In [13]:
import glob
os.chdir(caminho)
arquivos2 = glob.glob('*.xlsx')


In [ ]:
Juntar arquivos em um dataframe

In [14]:
arquivos2

['RMA_CRAS_Criterios_2022_divulga%C3%A7%C3%A3o_27_03_23.xlsx',
 'RMA_CRAS_Criterios_2023_divulgaç╞o_030423.xlsx']

Abrir apenas uma aba especifica do XLSX

In [15]:
tabelas=[]

for x in arquivos2:
    dados = pd.ExcelFile(x)
    print(f'Carregando arquivo {dados} ...')
    tabelas.append(pd.read_excel(dados,sheet_name="Base tratada"))

print('fim')
   
    

Carregando arquivo <pandas.io.excel._base.ExcelFile object at 0x0000024CA5B97B10> ...
Carregando arquivo <pandas.io.excel._base.ExcelFile object at 0x0000024CA57C40D0> ...
fim


In [16]:
len(tabelas)

2

In [17]:
tabela_final=pd.concat(tabelas)

In [18]:
tabela_final.head(2)

,data,mes_referencia,ano,mes,nome_unidade,NU_IDENTIFICADOR,endereco,municipio,uf,a1,...,d2_original,d3_original,d4_original,d5_original,d6_original,d7_original,d8_original,geral,Ind_duplicidade,trimestre
0,2022-01-01,01-01-2022,2022,1,'CRAS CENTRO DE REFERENCIA DE ASSISTENCIA SOCIAL','11000120396','RUA MARANHAO 3300 - PRINCESA ISABEL','ALTA FLORESTA DOESTE','RO',30.0,...,0,0,0,0,0,0,0,1,Caso principal,1.0
1,2022-02-01,01-02-2022,2022,2,'CRAS CENTRO DE REFERENCIA DE ASSISTENCIA SOCIAL','11000120396','RUA MARANHAO 3300 - PRINCESA ISABEL','ALTA FLORESTA DOESTE','RO',51.0,...,0,0,0,90,31,0,0,1,Caso principal,1.0


In [19]:
tabela_final.columns

Index(['data', 'mes_referencia', 'ano', 'mes', 'nome_unidade',
       'NU_IDENTIFICADOR', 'endereco', 'municipio', 'uf', 'a1', 'a2', 'b1',
       'b2', 'b3', 'b4', 'b5', 'b6', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'd1',
       'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'IBGE', 'c7', 'c8', 'c9', 'd8',
       'IBGE7', 'Nome_UF', 'Região', 'Município', 'Pop_total_2010',
       'Pop_Urbana2010', 'Pop_Rural2010', 'areaKm2', 'Porte_pop2010',
       'a1_original', 'a2_original', 'b1_original', 'b2_original',
       'b3_original', 'b4_original', 'b5_original', 'b6_original',
       'c1_original', 'c2_original', 'c3_original', 'c4_original',
       'c5_original', 'c6_original', 'c7_original', 'c8_original',
       'c9_original', 'd1_original', 'd2_original', 'd3_original',
       'd4_original', 'd5_original', 'd6_original', 'd7_original',
       'd8_original', 'geral', 'Ind_duplicidade', 'trimestre'],
      dtype='object')

In [52]:
dados_municipio=tabela_final[['municipio', 'uf', 'IBGE','IBGE7','Nome_UF', 'Região', 'Município', 'Pop_total_2010', 'Pop_Urbana2010', 'Pop_Rural2010', 'areaKm2', 'Porte_pop2010']]

In [55]:
dados_municipio=dados_municipio.drop_duplicates(subset=['municipio','uf','IBGE','IBGE7'], keep='last')

In [56]:
dados_municipio

,municipio,uf,IBGE,IBGE7,Nome_UF,Região,Município,Pop_total_2010,Pop_Urbana2010,Pop_Rural2010,areaKm2,Porte_pop2010
835,'FEIJO','AC',120030,1200302,12,1,Feijó,32412,16636,15776,27975.427,2
2303,'BONFIM','RR',140015,1400159,14,1,Bonfim,10943,3711,7232,8095.421,1
2315,'CANTA','RR',140017,1400175,14,1,Cantá,13902,2257,11645,7664.831,1
2359,'IRACEMA','RR',140028,1400282,14,1,Iracema,8696,4078,4618,14351.133,1
2397,'RORAINOPOLIS','RR',140047,1400472,14,1,Rorainópolis,24279,10673,13606,33596.525,2
...,...,...,...,...,...,...,...,...,...,...,...,...
7835,'VIANOPOLIS','GO',522200,5222005,52,5,Vianópolis,12548,9170,3378,954.284,1
7836,'VICENTINOPOLIS','GO',522205,5222054,52,5,Vicentinópolis,7371,6321,1050,737.255,1
7837,'VILA BOA','GO',522220,5222203,52,5,Vila Boa,4735,3502,1233,1060.172,1
7839,'VILA PROPICIO','GO',522230,5222302,52,5,Vila Propício,5145,1504,3641,2181.583,1


Porte do municipio

In [57]:
url='https://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv'
porte=pd.read_csv(url, sep=";", encoding='Latin-1')

In [72]:
porte=porte[['IBGE',	'IBGE7',	'UF',	'Município',	'Região',	'População 2010',	'Porte']	]

In [73]:
Dados_municipio_porte = pd.merge(porte, dados_municipio, how = 'left', right_on='IBGE7',left_on='IBGE7')

In [79]:
Dados_municipio_porte.sort_values(by="IBGE_y")

,IBGE_x,IBGE7,UF,Município_x,Região_x,População 2010,Porte,municipio,uf,IBGE_y,Nome_UF,Região_y,Município_y,Pop_total_2010,Pop_Urbana2010,Pop_Rural2010,areaKm2,Porte_pop2010
0,110001,1100015,RO,Alta Floresta D´oeste,Região Norte,24392.0,Pequeno II,'ALTA FLORESTA DOESTE','RO',110001.0,11.0,1.0,Alta Floresta D'Oeste,24392.0,13970.0,10422.0,7067.025,2.0
1,110002,1100023,RO,Ariquemes,Região Norte,90353.0,Médio,'ARIQUEMES','RO',110002.0,11.0,1.0,Ariquemes,90353.0,76525.0,13828.0,4426.571,3.0
2,110003,1100031,RO,Cabixi,Região Norte,6313.0,Pequeno I,'CABIXI','RO',110003.0,11.0,1.0,Cabixi,6313.0,2693.0,3620.0,1314.353,1.0
3,110004,1100049,RO,Cacoal,Região Norte,78574.0,Médio,'CACOAL','RO',110004.0,11.0,1.0,Cacoal,78574.0,61921.0,16653.0,3792.948,3.0
4,110005,1100056,RO,Cerejeiras,Região Norte,17029.0,Pequeno I,'CEREJEIRAS','RO',110005.0,11.0,1.0,Cerejeiras,17029.0,14419.0,2610.0,2783.300,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973,431700,4317004,RS,Santana da Boa Vista,Região Sul,8242.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4976,431725,4317251,RS,Santa Tereza,Região Sul,1720.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5056,432149,4321493,RS,Toropi,Região Sul,2952.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5076,432234,4322343,RS,Ubiretama,Região Sul,2296.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Municipios não encontrados no RMA 2022

In [198]:
MuncipiosFalta=Dados_municipio_porte[Dados_municipio_porte['IBGE_y'].isna()]

In [199]:
MuncipiosFalta

,IBGE_x,IBGE7,UF,Município_x,Região_x,População 2010,Porte,municipio,uf,IBGE_y,Nome_UF,Região_y,Município_y,Pop_total_2010,Pop_Urbana2010,Pop_Rural2010,areaKm2,Porte_pop2010
575,210730,2107308,MA,Nova Iorque,Região Nordeste,4590.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605,210945,2109452,MA,Raposa,Região Nordeste,26327.0,Pequeno II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,220196,2201960,PI,Brasileira,Região Nordeste,7966.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1662,270130,2701308,AL,Cajueiro,Região Nordeste,20409.0,Pequeno II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2230,293280,2932804,BA,Utinga,Região Nordeste,18173.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2356,311040,3110400,MG,Camacho,Região Sudeste,3154.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2709,314040,3140407,MG,Marmelópolis,Região Sudeste,2968.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3292,350230,3502309,SP,Anhembi,Região Sudeste,5653.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3342,350670,3506706,SP,Boa Esperança do Sul,Região Sudeste,13645.0,Pequeno I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3364,350860,3508603,SP,Cachoeira Paulista,Região Sudeste,30091.0,Pequeno II,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
MuncipiosFalta.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 575 to 5092
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   IBGE_x          50 non-null     int64  
 1   IBGE7           50 non-null     int64  
 2   UF              50 non-null     object 
 3   Município_x     50 non-null     object 
 4   Região_x        50 non-null     object 
 5   População 2010  49 non-null     float64
 6   Porte           50 non-null     object 
 7   municipio       0 non-null      object 
 8   uf              0 non-null      object 
 9   IBGE_y          0 non-null      float64
 10  Nome_UF         0 non-null      float64
 11  Região_y        0 non-null      float64
 12  Município_y     0 non-null      object 
 13  Pop_total_2010  0 non-null      float64
 14  Pop_Urbana2010  0 non-null      float64
 15  Pop_Rural2010   0 non-null      float64
 16  areaKm2         0 non-null      float64
 17  Porte_pop2010   0 non-null      float6

In [21]:
RMA_mensal.head(2)

,data,ano,mes,municipio,uf,c1,c2,c3,c4,c5,...,IBGE,IBGE7,Nome_UF,Região,Município,Pop_total_2010,Pop_Urbana2010,Pop_Rural2010,areaKm2,Porte_pop2010
0,2022-01-01,2022,1,'ALTA FLORESTA DOESTE','RO',1218.0,47.0,139.0,24.0,0.0,...,110001,1100015,11,1,Alta Floresta D'Oeste,24392,13970,10422,7067.025,2
1,2022-02-01,2022,2,'ALTA FLORESTA DOESTE','RO',1280.0,39.0,74.0,6.0,1.0,...,110001,1100015,11,1,Alta Floresta D'Oeste,24392,13970,10422,7067.025,2


In [115]:
RMA_mensal=tabela_final[['data', 'ano', 'mes', 'NU_IDENTIFICADOR','municipio', 'uf', 'c1', 'IBGE','IBGE7']]

In [116]:
RMA_mensal.info()


<class 'pandas.core.frame.DataFrame'>
Index: 107017 entries, 0 to 7862
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   data              107017 non-null  object 
 1   ano               107017 non-null  int64  
 2   mes               107017 non-null  int64  
 3   NU_IDENTIFICADOR  107017 non-null  object 
 4   municipio         107017 non-null  object 
 5   uf                107017 non-null  object 
 6   c1                104526 non-null  float64
 7   IBGE              107017 non-null  int64  
 8   IBGE7             107017 non-null  int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 8.2+ MB


In [123]:
RMA_m_2022=RMA_mensal[(RMA_mensal['ano']==2022)]

In [175]:
RMA_m_2022[0:13]

,data,ano,mes,NU_IDENTIFICADOR,municipio,uf,c1,IBGE,IBGE7
0,2022-01-01,2022,1,'11000120396','ALTA FLORESTA DOESTE','RO',1218.0,110001,1100015
1,2022-02-01,2022,2,'11000120396','ALTA FLORESTA DOESTE','RO',1280.0,110001,1100015
2,2022-03-01,2022,3,'11000120396','ALTA FLORESTA DOESTE','RO',1006.0,110001,1100015
3,2022-04-01,2022,4,'11000120396','ALTA FLORESTA DOESTE','RO',1116.0,110001,1100015
4,2022-05-01,2022,5,'11000120396','ALTA FLORESTA DOESTE','RO',1442.0,110001,1100015
5,2022-06-01,2022,6,'11000120396','ALTA FLORESTA DOESTE','RO',1057.0,110001,1100015
6,2022-07-01,2022,7,'11000120396','ALTA FLORESTA DOESTE','RO',1485.0,110001,1100015
7,2022-08-01,2022,8,'11000120396','ALTA FLORESTA DOESTE','RO',1802.0,110001,1100015
8,2022-09-01,2022,9,'11000120396','ALTA FLORESTA DOESTE','RO',1283.0,110001,1100015
9,2022-10-01,2022,10,'11000120396','ALTA FLORESTA DOESTE','RO',1033.0,110001,1100015


In [176]:
RMA_2022=RMA_m_2022.groupby(['ano','municipio','uf','IBGE','IBGE7'])['c1'].agg(['sum','mean']).reset_index()
RMA_2022=RMA_2022.rename(columns={'sum':'Soma_C1', 'mean': 'Média_C1'})

Check Consistencia do Municipio 2918357

In [178]:
RMA_2022[(RMA_2022['IBGE7']==2918357)]

,ano,municipio,uf,IBGE,IBGE7,Soma_C1,Média_C1
2565,2022,'JOAO DOURADO','BA',291835,2918357,24252.0,1276.421053


In [179]:
RMA_m_2022[(RMA_m_2022['IBGE7']==2918357)]

,data,ano,mes,NU_IDENTIFICADOR,municipio,uf,c1,IBGE,IBGE7
35641,2022-01-01,2022,1,'29183502715','JOAO DOURADO','BA',632.0,291835,2918357
35642,2022-02-01,2022,2,'29183502715','JOAO DOURADO','BA',857.0,291835,2918357
35643,2022-03-01,2022,3,'29183502715','JOAO DOURADO','BA',NaN,291835,2918357
35644,2022-04-01,2022,4,'29183502715','JOAO DOURADO','BA',1010.0,291835,2918357
35645,2022-05-01,2022,5,'29183502715','JOAO DOURADO','BA',1674.0,291835,2918357
35646,2022-06-01,2022,6,'29183502715','JOAO DOURADO','BA',860.0,291835,2918357
35647,2022-07-01,2022,7,'29183502715','JOAO DOURADO','BA',1495.0,291835,2918357
35648,2022-08-01,2022,8,'29183502715','JOAO DOURADO','BA',1725.0,291835,2918357
35649,2022-09-01,2022,9,'29183502715','JOAO DOURADO','BA',NaN,291835,2918357
35650,2022-10-01,2022,10,'29183502715','JOAO DOURADO','BA',1182.0,291835,2918357


In [244]:
RMA_m_2022[(RMA_m_2022['IBGE7']==2803302)]

,data,ano,mes,NU_IDENTIFICADOR,municipio,uf,c1,IBGE,IBGE7
31297,2022-01-01,2022,1,'28033003857','JAPARATUBA','SE',722.0,280330,2803302
31298,2022-02-01,2022,2,'28033003857','JAPARATUBA','SE',386.0,280330,2803302
31299,2022-03-01,2022,3,'28033003857','JAPARATUBA','SE',494.0,280330,2803302
31300,2022-04-01,2022,4,'28033003857','JAPARATUBA','SE',891.0,280330,2803302
31301,2022-05-01,2022,5,'28033003857','JAPARATUBA','SE',495.0,280330,2803302
31302,2022-06-01,2022,6,'28033003857','JAPARATUBA','SE',338.0,280330,2803302
31303,2022-07-01,2022,7,'28033003857','JAPARATUBA','SE',637.0,280330,2803302
31304,2022-08-01,2022,8,'28033003857','JAPARATUBA','SE',641.0,280330,2803302
31305,2022-09-01,2022,9,'28033003857','JAPARATUBA','SE',567.0,280330,2803302
31306,2022-10-01,2022,10,'28033003857','JAPARATUBA','SE',565.0,280330,2803302


In [180]:
RMA_m_2022_CRAS=RMA_m_2022.groupby(['data','ano','municipio','uf','IBGE','IBGE7'])['NU_IDENTIFICADOR'].agg(['count']).reset_index()
RMA_m_2022_CRAS

,data,ano,municipio,uf,IBGE,IBGE7,count
0,2022-01-01,2022,'ABADIA DE GOIAS','GO',520005,5200050,1
1,2022-01-01,2022,'ABADIA DOS DOURADOS','MG',310010,3100104,1
2,2022-01-01,2022,'ABADIANIA','GO',520010,5200100,1
3,2022-01-01,2022,'ABAETE','MG',310020,3100203,1
4,2022-01-01,2022,'ABAETETUBA','PA',150010,1500107,8
...,...,...,...,...,...,...,...
64005,2022-12-01,2022,'XIQUE-XIQUE','BA',293360,2933604,2
64006,2022-12-01,2022,'ZABELE','PB',251740,2517407,1
64007,2022-12-01,2022,'ZACARIAS','SP',355715,3557154,1
64008,2022-12-01,2022,'ZE DOCA','MA',211400,2114007,2


In [191]:
RMA_2022_CRAS = RMA_m_2022_CRAS.groupby(['ano','municipio','uf','IBGE','IBGE7'])['count'].agg(['max']).reset_index()
RMA_2022_CRAS=RMA_2022_CRAS.rename(columns={'max':'total_max_CRAS'})

Consistencia numero de CRAS

In [192]:
RMA_m_2022_CRAS[(RMA_m_2022_CRAS['IBGE7']==2918357)]

,data,ano,municipio,uf,IBGE,IBGE7,count
2499,2022-01-01,2022,'JOAO DOURADO','BA',291835,2918357,1
7873,2022-02-01,2022,'JOAO DOURADO','BA',291835,2918357,1
13233,2022-03-01,2022,'JOAO DOURADO','BA',291835,2918357,1
18597,2022-04-01,2022,'JOAO DOURADO','BA',291835,2918357,2
23963,2022-05-01,2022,'JOAO DOURADO','BA',291835,2918357,2
29277,2022-06-01,2022,'JOAO DOURADO','BA',291835,2918357,2
34517,2022-07-01,2022,'JOAO DOURADO','BA',291835,2918357,2
39770,2022-08-01,2022,'JOAO DOURADO','BA',291835,2918357,2
45198,2022-09-01,2022,'JOAO DOURADO','BA',291835,2918357,2
50580,2022-10-01,2022,'JOAO DOURADO','BA',291835,2918357,2


In [193]:
RMA_2022_CRAS[(RMA_2022_CRAS['IBGE7']==2918357)]

,ano,municipio,uf,IBGE,IBGE7,total_max_CRAS
2565,2022,'JOAO DOURADO','BA',291835,2918357,2


In [194]:
RMAcompleto = pd.merge(RMA_2022, RMA_2022_CRAS, how = 'inner')

In [195]:
RMAcompleto

,ano,municipio,uf,IBGE,IBGE7,Soma_C1,Média_C1,total_max_CRAS
0,2022,'ABADIA DE GOIAS','GO',520005,5200050,6860.0,857.500000,1
1,2022,'ABADIA DOS DOURADOS','MG',310010,3100104,1441.0,120.083333,1
2,2022,'ABADIANIA','GO',520010,5200100,1869.0,155.750000,1
3,2022,'ABAETE','MG',310020,3100203,6640.0,553.333333,1
4,2022,'ABAETETUBA','PA',150010,1500107,26806.0,279.229167,8
...,...,...,...,...,...,...,...,...
5513,2022,'XIQUE-XIQUE','BA',293360,2933604,4144.0,172.666667,2
5514,2022,'ZABELE','PB',251740,2517407,961.0,80.083333,1
5515,2022,'ZACARIAS','SP',355715,3557154,922.0,76.833333,1
5516,2022,'ZE DOCA','MA',211400,2114007,4779.0,796.500000,2


In [196]:
RMAcompleto.sort_values(by=['Soma_C1'], ascending=False)

,ano,municipio,uf,IBGE,IBGE7,Soma_C1,Média_C1,total_max_CRAS
4166,2022,'RIO DE JANEIRO','RJ',330455,3304557,1455179.0,3156.570499,47
4809,2022,'SAO PAULO','SP',355030,3550308,901863.0,1426.998418,54
1818,2022,'FORTALEZA','CE',230440,2304400,738816.0,2556.456747,27
1493,2022,'CURITIBA','PR',410690,4106902,508362.0,1086.243590,39
2856,2022,'MANAUS','AM',130260,1302603,350503.0,1466.539749,20
...,...,...,...,...,...,...,...,...
1551,2022,'DIRCEU ARCOVERDE','PI',220335,2203354,0.0,NaN,1
4519,2022,'SANTO ANTONIO DO SUDOESTE','PR',412440,4124400,0.0,NaN,1
4125,2022,'RIBEIRAO BONITO','SP',354290,3542909,0.0,0.000000,1
5173,2022,'TERRA ROXA','PR',412740,4127403,0.0,NaN,1


In [211]:
RMAcompleto_sem_Nulos=RMAcompleto[(RMAcompleto['Soma_C1'] != 0 )].sort_values(by=['Soma_C1'], ascending=False)


In [212]:
RMAcompleto_sem_Nulos

,ano,municipio,uf,IBGE,IBGE7,Soma_C1,Média_C1,total_max_CRAS
4166,2022,'RIO DE JANEIRO','RJ',330455,3304557,1455179.0,3156.570499,47
4809,2022,'SAO PAULO','SP',355030,3550308,901863.0,1426.998418,54
1818,2022,'FORTALEZA','CE',230440,2304400,738816.0,2556.456747,27
1493,2022,'CURITIBA','PR',410690,4106902,508362.0,1086.243590,39
2856,2022,'MANAUS','AM',130260,1302603,350503.0,1466.539749,20
...,...,...,...,...,...,...,...,...
2412,2022,'ITOBI','SP',352380,3523800,42.0,8.400000,1
676,2022,'BOM JESUS DO OESTE','SC',420257,4202578,36.0,36.000000,1
4370,2022,'SANTA JULIANA','MG',315770,3157708,35.0,7.000000,1
5052,2022,'SULINA','PR',412665,4126652,32.0,32.000000,1


In [213]:
RMAcompleto_Nulos=RMAcompleto[(RMAcompleto['Soma_C1'] == 0 )].sort_values(by=['Soma_C1'], ascending=False)

In [214]:
RMAcompleto_Nulos

,ano,municipio,uf,IBGE,IBGE7,Soma_C1,Média_C1,total_max_CRAS
726,2022,'BOTELHOS','MG',310840,3108404,0.0,NaN,1
3071,2022,'MISSAL','PR',411605,4116059,0.0,NaN,1
4519,2022,'SANTO ANTONIO DO SUDOESTE','PR',412440,4124400,0.0,NaN,1
4125,2022,'RIBEIRAO BONITO','SP',354290,3542909,0.0,0.0,1
3764,2022,'PINDORETAMA','CE',231085,2310852,0.0,NaN,1
3644,2022,'PAVERAMA','RS',431415,4314159,0.0,NaN,1
3242,2022,'NONOAI','RS',431270,4312708,0.0,NaN,1
3206,2022,'NAQUE','MG',314435,3144359,0.0,NaN,1
2863,2022,'MANGA','MG',313930,3139300,0.0,NaN,1
920,2022,'CAMOCIM DE SAO FELIX','PE',260350,2603504,0.0,NaN,1


In [215]:
RMAcompleto_sem_Nulos               

,ano,municipio,uf,IBGE,IBGE7,Soma_C1,Média_C1,total_max_CRAS
4166,2022,'RIO DE JANEIRO','RJ',330455,3304557,1455179.0,3156.570499,47
4809,2022,'SAO PAULO','SP',355030,3550308,901863.0,1426.998418,54
1818,2022,'FORTALEZA','CE',230440,2304400,738816.0,2556.456747,27
1493,2022,'CURITIBA','PR',410690,4106902,508362.0,1086.243590,39
2856,2022,'MANAUS','AM',130260,1302603,350503.0,1466.539749,20
...,...,...,...,...,...,...,...,...
2412,2022,'ITOBI','SP',352380,3523800,42.0,8.400000,1
676,2022,'BOM JESUS DO OESTE','SC',420257,4202578,36.0,36.000000,1
4370,2022,'SANTA JULIANA','MG',315770,3157708,35.0,7.000000,1
5052,2022,'SULINA','PR',412665,4126652,32.0,32.000000,1


In [217]:
RMA_2022_porte=pd.merge(RMAcompleto_sem_Nulos,Dados_municipio_porte, how = 'inner', right_on='IBGE7',left_on='IBGE7')

In [218]:
RMA_2022_porte.columns

Index(['ano', 'municipio_x', 'uf_x', 'IBGE', 'IBGE7', 'Soma_C1', 'Média_C1',
       'total_max_CRAS', 'IBGE_x', 'UF', 'Município_x', 'Região_x',
       'População 2010', 'Porte', 'municipio_y', 'uf_y', 'IBGE_y', 'Nome_UF',
       'Região_y', 'Município_y', 'Pop_total_2010', 'Pop_Urbana2010',
       'Pop_Rural2010', 'areaKm2', 'Porte_pop2010'],
      dtype='object')

In [246]:
RMA_TOTAL_2022=RMA_2022_porte[['ano', 'IBGE', 'IBGE7', 'Município_y', 'uf_y','Soma_C1', 'Média_C1','total_max_CRAS', 'Porte']]

In [252]:
RMA_TOTAL_2022=RMA_TOTAL_2022.rename(columns={'IBGE':'ibge_6',	'IBGE7':'ibge_7','Município_y':'municipio','uf_y':'uf',	'Soma_C1':'soma_c1',
                               'Média_C1':'media_c1','total_max_CRAS':'total_max_cras','Porte':'porte'})

In [253]:
RMA_TOTAL_2022

,ano,ibge_6,ibge_7,municipio,uf,soma_c1,media_c1,total_max_cras,porte
0,2022,330455,3304557,Rio de Janeiro,'RJ',1455179.0,3156.570499,47,Metrópole
1,2022,355030,3550308,São Paulo,'SP',901863.0,1426.998418,54,Metrópole
2,2022,230440,2304400,Fortaleza,'CE',738816.0,2556.456747,27,Metrópole
3,2022,410690,4106902,Curitiba,'PR',508362.0,1086.243590,39,Metrópole
4,2022,130260,1302603,Manaus,'AM',350503.0,1466.539749,20,Metrópole
...,...,...,...,...,...,...,...,...,...
5496,2022,352380,3523800,Itobi,'SP',42.0,8.400000,1,Pequeno I
5497,2022,420257,4202578,Bom Jesus do Oeste,'SC',36.0,36.000000,1,Pequeno I
5498,2022,315770,3157708,Santa Juliana,'MG',35.0,7.000000,1,Pequeno I
5499,2022,412665,4126652,Sulina,'PR',32.0,32.000000,1,Pequeno I


10 maiores RMA

In [254]:
RMA_TOTAL_2022.sort_values(by=['soma_c1'], ascending=False)[0:10]

,ano,ibge_6,ibge_7,municipio,uf,soma_c1,media_c1,total_max_cras,porte
0,2022,330455,3304557,Rio de Janeiro,'RJ',1455179.0,3156.570499,47,Metrópole
1,2022,355030,3550308,São Paulo,'SP',901863.0,1426.998418,54,Metrópole
2,2022,230440,2304400,Fortaleza,'CE',738816.0,2556.456747,27,Metrópole
3,2022,410690,4106902,Curitiba,'PR',508362.0,1086.243590,39,Metrópole
4,2022,130260,1302603,Manaus,'AM',350503.0,1466.539749,20,Metrópole
5,2022,221100,2211001,Teresina,'PI',322554.0,1688.764398,19,Grande
6,2022,500270,5002704,Campo Grande,'MS',309265.0,1338.809524,21,Grande
7,2022,351880,3518800,Guarulhos,'SP',305394.0,2229.153285,12,Metrópole
8,2022,211130,2111300,São Luís,'MA',300974.0,1308.582609,20,Metrópole
9,2022,530010,5300108,Brasília,'DF',300262.0,1042.576389,29,Metrópole


10 menores RMA

In [255]:
RMA_TOTAL_2022.sort_values(by=['soma_c1'], ascending=True)[0:10]

,ano,ibge_6,ibge_7,municipio,uf,soma_c1,media_c1,total_max_cras,porte
5500,2022,354250,3542503,Reginópolis,'SP',13.0,2.600000,1,Pequeno I
5499,2022,412665,4126652,Sulina,'PR',32.0,32.000000,1,Pequeno I
5498,2022,315770,3157708,Santa Juliana,'MG',35.0,7.000000,1,Pequeno I
5497,2022,420257,4202578,Bom Jesus do Oeste,'SC',36.0,36.000000,1,Pequeno I
5496,2022,352380,3523800,Itobi,'SP',42.0,8.400000,1,Pequeno I
5495,2022,270660,2706604,Paulo Jacinto,'AL',44.0,6.285714,1,Pequeno I
5493,2022,220920,2209203,Santa Filomena,'PI',50.0,50.000000,1,Pequeno I
5494,2022,280430,2804300,Muribeca,'SE',50.0,50.000000,1,Pequeno I
5491,2022,352770,3527702,Luiziânia,'SP',56.0,14.000000,1,Pequeno I
5492,2022,270600,2706000,Olivença,'AL',56.0,56.000000,1,Pequeno I


In [258]:
RMA_TOTAL_2022.to_csv('RMA_TOTAL_2022.csv',encoding = 'UTF-8')